In [ ]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
import argparse

from dataset_paths import *

warnings.filterwarnings('ignore')

output_version = "20241229"
outdir = "/media/hieunguyen/HNSD_mini/outdir"
dataset_name = "LOD"
mode = "all"
PROJECT = "combine_ctcandi_ichorcna"
input_cancer_class = "pan_cancer"

path_to_main_output = os.path.join(outdir, PROJECT, output_version, dataset_name)
path_to_07_output = os.path.join(path_to_main_output, f"07_output_{mode}", input_cancer_class)
path_to_08_output = os.path.join(path_to_main_output, f"08_output_{mode}", input_cancer_class)
os.system(f"mkdir -p {path_to_08_output}")
all_read_classification_files = [item for item in pathlib.Path(path_to_07_output).glob("*.raw.read_classification.csv")]
all_candi_read_files = [item for item in pathlib.Path(path_to_07_output).glob("*candi_reads.csv")]

metadata = pd.read_csv(f"./metadata/{dataset_name}.csv")
metadata = metadata[metadata["Sample"].duplicated() == False]

if os.path.isfile(os.path.join(path_to_08_output, "feature.csv")) == False:
    fulldf = pd.DataFrame()
    for i in tqdm(range(len(all_candi_read_files))):
        tmp_readdf = pd.read_csv(all_read_classification_files[i], index_col = [0])
        tmp_candidf = pd.read_csv(all_candi_read_files[i], index_col = [0])
        sampleid = all_candi_read_files[i].name.replace(".candi_reads.csv", "")
        raw_count = tmp_readdf.shape[0]
        in_read_count = tmp_readdf[tmp_readdf["read_overlap_rate"] == "in"].shape[0]
        mean_candi_reads = tmp_candidf.candi_reads.mean()
        ratio_raw = mean_candi_reads/raw_count
        ratio_in_read = mean_candi_reads/in_read_count
        tmpdf = pd.DataFrame({"SampleID": sampleid, 
                            "raw_count": raw_count, 
                            "in_read_count": in_read_count, 
                            "mean_candi_reads": mean_candi_reads,
                            "ratio_raw": ratio_raw,
                            "ratio_in_read": ratio_in_read}, index = [0])

        fulldf = pd.concat([fulldf, tmpdf], axis = 0)
    fulldf.merge(metadata, left_on = "SampleID", right_on = "Sample")
    fulldf.to_csv(os.path.join(path_to_08_output, "feature.csv"))
else:
    fulldf = pd.read_csv(os.path.join(path_to_08_output, "feature.csv"), index_col = [0])


  7%|▋         | 30/427 [00:03<00:50,  7.92it/s]

,sampleid,raw_count,in_read_count,mean_candi_reads,ratio_raw,ratio_in_read,Sample,Actual tumor_fraction_ichorCNA,Simulated TF,spike-in,LABEL,LABCODE,Detected,Decision,RUNTM,RUNGW
0,LODLIV12,18031,9085,4.684366,0.000260,0.000516,LODLIV12,0.05390,0.03017,5,Liver cancer,LIVER5,Yes,Cancer,R3703,R3703
1,LODBREAST18R2,19666,10266,4.967262,0.000253,0.000484,LODBREAST18R2,0.02083,0.0010415,5,Breast cancer,BREAST9,No,Healthy,R3724,R3725
2,LODCONTROL3,12360,6358,2.975535,0.000241,0.000468,LODCONTROL3,0.02966,Healthy-control,HC,Colorectal cancer,NaN,No,NaN,R3703,R3703
3,LODCONTROL3,12360,6358,2.975535,0.000241,0.000468,LODCONTROL3,0.02966,Healthy-control,HC,Lung cancer,NaN,No,NaN,R3703,R3703
4,LODCONTROL3,12360,6358,2.975535,0.000241,0.000468,LODCONTROL3,0.02966,Healthy-control,HC,Liver cancer,NaN,No,NaN,R3703,R3703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,CONTROL31R2,9162,4580,2.192926,0.000239,0.000479,CONTROL31R2,0.03182,Healthy-control,HC,Breast cancer,NaN,No,NaN,R3354,R3354
675,CONTROL31R2,9162,4580,2.192926,0.000239,0.000479,CONTROL31R2,0.03182,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3354,R3354
676,LODLUNG19R2,26695,13462,6.557864,0.000246,0.000487,LODLUNG19R2,0.02323,0.0093105,15,Lung cancer,LUNG15,No,Healthy,R3725,R3724
677,GASTRIC2,19746,10215,5.183976,0.000263,0.000507,GASTRIC2,0.02463,0.02463,100,Gastric cancer,GASTRIC2,No,Cancer,R3666,R3666


In [47]:
metadata[metadata["Sample"].isin(fulldf["sampleid"].unique())]

,Sample,Actual tumor_fraction_ichorCNA,Simulated TF,spike-in,LABEL,LABCODE,Detected,Decision,RUNTM,RUNGW
0,LODCRC6,0.02443,0.003226,1,Colorectal cancer,CRC8,No,Healthy,R3596,R3606
1,BREAST10R1,0.02187,0.02187,100,Breast cancer,BREAST10,No,Healthy,R3725,R3724
2,LODCRC3PR1,0.07539,0.056745,15,Colorectal cancer,CRC8,No,Healthy,R3549,R3549
3,LODBREAST7R2,0.02290,0.0001145,0.5,Breast cancer,BREAST7,No,Healthy,R3596,R3606
4,LODBREAST6,0.02238,0.0006958,1,Breast cancer,BREAST7,No,Healthy,R3596,R3606
...,...,...,...,...,...,...,...,...,...,...
674,CONTROL44R9,0.03381,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3715,R3715
675,LODCONTROL2,0.02951,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3683,R3683
676,LODCONTROL3,0.02966,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3703,R3703
677,LODCONTROL4R1,0.04060,Healthy-control,HC,Gastric cancer,NaN,No,NaN,R3715,R3715
